<a href="https://colab.research.google.com/github/kmurdzek/HousingData/blob/main/HousingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
!pip install --upgrade google-cloud-bigquery
!export GOOGLE_APPLICATION_CREDENTIALS='/content/housing-project-351117-3c3b2a57bb4d.json'
#https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries

In [ ]:

project_id = 'housing-project-351117'
from google.cloud import bigquery
sample_count = 2000
client = bigquery.Client(project=project_id)
# make queries to identify useful data
# get the average home sale price per year
# get the average home sale price of homes of a particular square footage per year
choice = input("Choose a following option by entering the number associated with the option.\n 1. Get the average home sale price in a given year.\n 2. Get the average home sale price for a house with given number of beds/baths.\n 3. Provide your budget to see what kind of homes you'd be able to afford.\n")

if choice == "1":
  year = input("Enter the year you want the average home sale price for : ")
  sql = "SELECT AVG(SalePrice) as AverageSale FROM `housing-project-351117.housing.housing_data` WHERE YrSold = {}".format(year)
  average_sale_price = client.query(sql)
  result = average_sale_price.result()
  for row in result:
    if row.AverageSale is None:
      print("Unable to find provide an average for the given year")
      break
    formatted = "${:,.2f}".format(row.AverageSale)
    print("Average sale price in {}: {}".format(year, formatted))
elif(choice == "2"):
  bedrooms = input("Enter the desired minimum number of bedrooms: \n")
  bathrooms = input("Enter the desired minimum number of bathrooms: \n")
  sql = "SELECT Avg(SalePrice) as AverageSale FROM `housing-project-351117.housing.housing_data` WHERE FullBath + HalfBath >= {} AND BedroomAbvGr >= {}".format(bathrooms, bedrooms);
  average_sale_price = client.query(sql)
  result = average_sale_price.result()
  for row in result:
    if row.AverageSale is None:
      print("No data fits the entered criteria")
      break
    formatted = "${:,.2f}".format(row.AverageSale)
    print("Average sale price of home with atleast {} bedrooms and {} bathrooms: {}".format(bedrooms, bathrooms, formatted))
elif choice == "3":
  budget = input("Enter your budget: \n")
  bedrooms = input("Enter the desired minimum number of bedrooms: \n")
  bathrooms = input("Enter the desired minimum number of bathrooms: \n")
  sql = "SELECT Id, SalePrice, HalfBath, FullBath, BedroomAbvGr, LotArea, YrSold FROM `housing-project-351117.housing.housing_data` WHERE SalePrice < {} AND HalfBath + FullBath >= {} AND BedroomAbvGr >= {}".format(budget, bedrooms, bathrooms)
  data = client.query(sql)
  result = data.result()
  for row in result:
    if row.Id is None:
      print("Unable to find houses with your given budget or criteria")
      break
    formatted = "${:,.2f}".format(row.SalePrice)
    totalBathrooms = row.HalfBath + row.FullBath
    print("House ID: {}, Sale Price: {}, Number of Bathrooms {}, Number of Bedrooms {}, Lot Size {}, Year sold {}". format(row.Id, formatted, totalBathrooms, row.BedroomAbvGr, row.LotArea, row.YrSold))
  option = input("Would you like the average values for each of the above data points?\n 1. yes \t 2. no\n")
  if option == "1":
    sql = "SELECT AVG(SalePrice) as averageSale,AVG(HalfBath) as averageHalfBath, AVG(FullBath) as averageFullBath, AVG(BedroomAbvGr) as bedroomAvg, AVG(LotArea) as lotAvg FROM `housing-project-351117.housing.housing_data` WHERE SalePrice < {} AND HalfBath + FullBath >= {} AND BedroomAbvGr >= {}".format(budget, bedrooms, bathrooms)
    data = client.query(sql)
    result = data.result()
    for row in result:
      if row.averageSale is None:
        print("Data cannot be gathered")
      formatted = "${:,.2f}".format(row.averageSale)
      totalBathroomAverage = row.averageHalfBath + row.averageFullBath
      print("The average sale price with your criteria is: {}, Average Number of Bathrooms: {}, Average Number of Bedrooms: {}, Average Lot Area: {} sq ft".format(formatted, round(totalBathroomAverage), round(row.bedroomAvg), round(row.lotAvg)))
  else:
    pass
else:
  pass
  print('Invalid Option')
